![MSE Logo](https://moodle.msengineering.ch/pluginfile.php/1/core_admin/logo/0x150/1643104191/logo-mse.png)

# AnTeDe Lab 4: Search Engine with the Vector Space Model

## Summary
The aim of this lab is to build a simple document search engine based on TF-IDF document vectors. 

The lab is inspired by a notebook designed by [Kavita Ganesan](https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb).

<font color='green'>Please answer the questions in green within this notebook, and submit the completed notebook under the corresponding homework on Moodle.</font>

In [2]:
!pip install contractions
import os    
import nltk  # on Colab, you mind find it helpful to run nltk.download('popular') to install packages
import gensim
import pandas as pd
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel
from gensim import models, corpora, similarities

     |████████████████████████████████| 106 kB 34.2 MB/s 
     |████████████████████████████████| 284 kB 46.2 MB/s 


2022-03-20 19:12:16,166 : INFO : NumExpr defaulting to 2 threads.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

# Modify path according to your configuration
# !ls "/content/gdrive/MyDrive/ColabNotebooks/MSE_AnTeDe_Spring2022"
import sys
sys.path.insert(0,'/content/gdrive/MyDrive/Colab Notebooks/MSE/AnTeDe/MSE_AnTeDe_Lab4')

from TextPreprocessor import *

Mounted at /content/gdrive


The data used in this lab is a set of 300 documents selected from the Australian Broadcasting Corporation's news mail service. It consists of texts of headline stories from around the years 2000-2001.  This is a shortened version of the Lee Background Corpus [described here](http://www.socsci.uci.edu/~mdlee/lee_pincombe_welsh_document.PDF).  It is available as test data in the **gensim** package, so you do not need to download it separately.

The following code will load the documents into a Pandas dataframe.

In [4]:
# Code inspired from:
# https://github.com/bhargavvader/personal/blob/master/notebooks/text_analysis_tutorial/topic_modelling.ipynb

test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
text = open(lee_train_file).read().splitlines()
data_df = pd.DataFrame({'text': text})

The following code will run our in-house Text Preprocessor provided in the `TextPreprocessor.py` file, and documented in the `MSE_AnTeDe_TextPreprocessingDemo.ipynb` notebook provided in Lab 1 (see Lab 1 archive on Moodle for both files).

<font color='green'>Please enrich the following code according your needs (especially stopwords)</font>

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
language = 'english'
stop_words = set(stopwords.words(language))
# Extend the list here:
# TextPreprocessor? - get help regarding the attributes

processor = TextPreprocessor(
# Add options here:
 language = language,
 stopwords = stop_words
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
data_df['processed'] = processor.transform(data_df['text'])

In [7]:
print(data_df['processed'].iloc[136])

new report suggests cost age australian population exaggerated report issue australia institute say detailed examination population health data show age population create unsustainable burden shrink workforce far economic social burden found majority old people enjoy healthy independent life many make financial contribution family participate voluntary community activity paper challenge assumption old population see health cost rise unsustainable level say rise health cost cause mainly factor age growth medical technology rise consumer demand escalate price


## Generation of document vectors with [Scikit-learn](https://scikit-learn.org/stable)

We will use the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) class from scikit-learn to create a vocabulary and generate word counts or *Term Frequencies* (TF).
    
The result is a  matrix representation of the counts: each column represents a _word_ in the vocabulary and each row represents a document in our dataset: the cell values are the word counts of the word in the document. 

The matrix is very sparse, because all words not appearing in a document have 0 counts.

In [8]:
cv = CountVectorizer(max_features=3000) # keep only the 3000 most frequent words in the corpus
word_count_vector = cv.fit_transform(data_df['processed'])

Let's look at some words from our vocabulary:

In [9]:
feature_names = cv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [45]:
print(len(feature_names)) # has the max_features value been reached?
print(feature_names[2700:2705]) # try various slices
print(feature_names.index('hundred')) # find a word

3000
['title', 'today', 'todd', 'toddler', 'together']
1264


**TfidfTransformer to Compute Inverse Document Frequency (IDF)**

We now use the (sparse) matrix generated by `CountVectorizer` to compute the IDF values of each word.  Note that the IDF should in reality be based on a large and representative corpus.

In [11]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

The IDF values are stored in the `idf_` field of the `TfidfTransformer`.  It has the same size as the array of feature names (words).

In [46]:
print(len(tfidf_transformer.idf_)) # check length
print(tfidf_transformer.idf_[cv.get_feature_names().index('hundred')]) # check IDF value of a word

3000
3.711377991194885


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


**We define here two helper functions:**
 * the first one is a sorting function for the columns of a sparse matrix in COOrdinate format (a.k.a "ijv" or "triplet" format [explained here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html));
 * the second one extracts the feature names (*words*) and their TF-IDF values from the sorted list.

In [13]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items from sorted list"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

We now select a document for which we will generate TF-IDF values.  <font color="green">Please select a random document of your choice between 0 and 300.</font>

In [48]:
# doc_orig = data_df['text'].iloc[136]
# doc_processed = data_df['processed'].iloc[136]
doc_orig = data_df['text'].iloc[139]
doc_processed = data_df['processed'].iloc[139]
print(doc_orig)
print(doc_processed)

Australia will be looking to score quickly today to set South Africa a challenging victory target on day four of the first cricket Test in Adelaide. The Australians will resume their second innings at 0 for 3, an overall lead of 68. South Africa was dismissed late yesterday for 374 with Shane Warne taking five wickets for the 20th time in his Test career. Warne says Australia is well placed to win. "I was very happy with our position in the match, 10 wickets in hand and 70 runs ahead on a pitch that's deteriorating. I think I'd much rather be in our shoes than theirs," he said. But he says Australia will need to bat well today. "South Africa can come out and bowl us out for 150 and suddenly they're chasing 200, or we can make 200 to 250 and they need 300. It's going to be a great last two days." 
australia look score quickly today set south africa challenge victory target day four first cricket test adelaide australian resume second inning overall lead 68. south africa dismiss late yes

**The next instruction generates the vector of TF-IDF values for the document** using the `tfidf_transformer`.

In [15]:
tf_idf_vector = tfidf_transformer.transform(cv.transform([doc_processed]))

Next, we sort the words in the `tf_idf_vector` by decreasing TF-IDF values, first transforming the vector into a coordinate format ('coo'), and then applying our sorting function from above.  We then extract the words with the top 10 scores (and the scores) for the selected document using our second helper function from above and display them.

In [16]:
sorted_items=sort_coo(tf_idf_vector.tocoo())

topn_words = extract_topn_from_vector(feature_names, sorted_items, 10)

print(doc_orig, '\n', topn_words)

Australia will be looking to score quickly today to set South Africa a challenging victory target on day four of the first cricket Test in Adelaide. The Australians will resume their second innings at 0 for 3, an overall lead of 68. South Africa was dismissed late yesterday for 374 with Shane Warne taking five wickets for the 20th time in his Test career. Warne says Australia is well placed to win. "I was very happy with our position in the match, 10 wickets in hand and 70 runs ahead on a pitch that's deteriorating. I think I'd much rather be in our shoes than theirs," he said. But he says Australia will need to bat well today. "South Africa can come out and bowl us out for 150 and suddenly they're chasing 200, or we can make 200 to 250 and they need 300. It's going to be a great last two days."  
 {'africa': 0.322, 'warne': 0.253, 'wicket': 0.235, 'south': 0.201, 'test': 0.19, 'australia': 0.188, 'need': 0.17, 'well': 0.155, 'suddenly': 0.153, 'deteriorate': 0.153}


<font color="green">Please comment briefly on the relevance of these words with respect to the document content.</font>

> The words with a higher tf-idf value are less common across all the documents but have a higher influence to the currently open document at index 139 (which was choosen by me)

## Document-document similarity using scikit-learn

In this section, you will write the commands to compute a document-document similarity matrix over the above documents, in scikit-learn.

Please use a processing [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) and a [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and compute the *cosine similarities* between all documents.  

<font color="green">At the end, you will be asked to display the five most similar documents to the one you selected above, and compare the 1st and the 5th best results.</font>

You can use inspiration from: 
 * the above code
 * https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XkK2ceFCe-Y
 * https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
 * https://stackoverflow.com/questions/12118720/python-tf-idf-cosine-to-find-document-similarity
 * https://markhneedham.com/blog/2016/07/27/scitkit-learn-tfidf-and-cosine-similarity-for-computer-science-papers

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfidf = TfidfVectorizer(use_idf=True)
pipe = Pipeline(steps=[('pre', processor), ('tfidf', tfidf)]) # the 'processor' was defined above

<font color='green'>Please write a function called `find_similar` which receives a `tfidf_matrix` with all similarity scores between documents, and the `index` of a document in the collection, and returns the `top_n` most similar documents to it using cosine similarity.</font>

In [19]:
def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    return cosine_similarities[index].argsort()[:-top_n-2:-1][1:]
    #return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]
    #indices = 
    #return indices

<font color="green">Using the data from the Pandas form created above, please use "fit" and "transform" to generate the matrix of all document similarites called "tfidf_matrix". -- How long do these two operations take on your computer?  -- Please explain briefly in your own words what is the difference between "fit" and "transform".</font>

In [21]:
import time
start = time.time()

tfidf_matrix = pipe.fit_transform(data_df['text'])

end = time.time()
print(f'transform and fit done in {end - start}')

print(tfidf_matrix.shape)

transform and fit done in 19.300696849822998
(300, 5551)


With `fit` the model is being created and with transform the vectorizer is used and applied to the data.

<font color="green">Using `find_similar` and the `tfidf_matrix` please display the five most similar documents to the one you selected above, with their scores, comment them, and compare the 1st and the 5th best results.</font>

In [22]:
find_similar(tfidf_matrix, 139, top_n = 5)

array([132, 182, 118, 112, 104])

<font color='green'>Could you also use the dot product instead of the cosine similarity in the `find_similar` function?  Please answer in the following box.</font>

In [23]:
'''
No - as it only compares if two given words are occuring together in different documents. It can not distinguish which word it is as it sums up the products of words
'''

'\nNo - as it only compares if two given words are occuring together in different documents. It can not distinguish which word it is as it sums up the products of words\n'

## Building a search engine using Gensim

<font color='green'>Using the [tutorial on Topics and Transformations from Gensim](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py), please implement a method that returns the documents most similar to a given query.
    
Use [Gensim's TF-IDF Model](https://radimrehurek.com/gensim/models/tfidfmodel.html) to build the model and the [MatrixSimilarity function](https://radimrehurek.com/gensim/similarities/docsim.html#gensim.similarities.docsim.MatrixSimilarity) to measure cosine similarity between documents.</font>

<font color='green'>Please write a query of your own (5-10 words), retrieve the 5 most similar documents, and comment the result.</font>

In [67]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import models
from gensim.utils import simple_preprocess

doc_tokenized = [simple_preprocess(doc) for doc in data_df['processed']]
dictionary = corpora.Dictionary(doc_tokenized)
corpus = [dictionary.doc2bow(text) for text in doc_tokenized]

model = models.TfidfModel(corpus)
print(model)

2022-03-20 20:56:26,951 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-20 20:56:27,031 : INFO : built Dictionary(5443 unique tokens: ['across', 'aedt', 'area', 'around', 'associate']...) from 300 documents (total 34244 corpus positions)
2022-03-20 20:56:27,097 : INFO : collecting document frequencies
2022-03-20 20:56:27,100 : INFO : PROGRESS: processing document #0
2022-03-20 20:56:27,114 : INFO : calculating IDF weights for 300 documents and 5442 features (25035 matrix non-zeros)


TfidfModel(num_docs=300, num_nnz=25035)


In [85]:
from gensim.similarities import MatrixSimilarity

doc = "I love sports very much but i don't like cricket."
vec_bow = dictionary.doc2bow(doc.lower().split())
sims = model[vec_bow]
index = MatrixSimilarity(model[corpus], num_best=5)

sims = index[sims]
print(sims)
for sim, similarity in sims:
    print(f'Similarity {similarity} {sim} : ' + data_df['processed'].iloc[sim])

2022-03-20 21:03:10,708 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2022-03-20 21:03:10,934 : INFO : creating matrix with 300 documents and 5443 features


[(112, 0.10040326416492462), (104, 0.10040326416492462), (146, 0.07662541419267654), (150, 0.07574578374624252), (156, 0.07574578374624252)]
Similarity 0.10040326416492462 112 : australian cricket captain steve waugh support fast bowler brett lee criticism intimidatory bowling south african tailenders first test adelaide earlier month lee fin give new zealand tailender shane bond send-off third test perth waugh say tailenders protect short-pitched bowling `` day earn big money get responsibility learn bat '' say `` mean time like year ago professional sort bowler code `` day professional batsman work hard batting expect tailenders likewise '' meanwhile waugh say side need guard complacency convincingly win first test run waugh say despite dominance side first test south africa never take lightly `` one test match three six whichever way want look lot work go '' say `` nice win first battle definitely give u lot confidence go melbourne know big crowd love play front boxing day crowd adv

## End of Lab 4
Please make sure all cells have been executed, save this completed notebook, compress it to a *zip* file, and upload it to [Moodle](https://moodle.msengineering.ch/course/view.php?id=1869).